In [ ]:
import os

In [ ]:
data_path = '/content/drive/MyDrive/Colab Notebooks/new mask detection/mask and no_mask'

categories = os.listdir(data_path)

labels = [i for i in range(len(categories))]

label_dict = dict(zip(categories, labels))

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
target_size = 224
data = []
label = []

for category in categories:
  folder_path = os.path.join(data_path, category)
  image_titles = os.listdir(folder_path)
  for name in image_titles:
    image_path = os.path.join(folder_path, name)
    image = cv2.imread(image_path)
    try:
      resize = cv2.resize(image, (target_size, target_size))

      data.append(resize)

      label.append(label_dict[category])
    except Exception as e:
      print('exception', e)

In [ ]:
import numpy
from keras.utils import np_utils

data = numpy.array(data)
Data = numpy.reshape(data, (data.shape[0], target_size, target_size, 3))
label = numpy.array(label)

Label = np_utils.to_categorical(label)
data.shape, Data.shape

((1006, 224, 224, 3), (1006, 224, 224, 3))

In [ ]:
from sklearn.model_selection import  train_test_split
x_train, x_test, y_train, y_test = train_test_split(Data, Label, test_size=0.05, random_state=2021)

In [ ]:
x_train = x_train/255.0
x_test = x_test/255.0

In [ ]:
from tensorflow import keras

model = keras.models.Sequential([
                                 keras.layers.Conv2D(512, (3,3), input_shape=(target_size, target_size, 3), activation='relu'),
                                 keras.layers.AvgPool2D(pool_size = (2,2)),

                                 keras.layers.Conv2D(512, (3,3), activation='relu'),
                                 keras.layers.AvgPool2D(pool_size = (2,2)),

                                 keras.layers.Conv2D(512, (3,3), activation='relu'),
                                 keras.layers.AvgPool2D(pool_size = (2,2)),

                                 keras.layers.Conv2D(256, (3,3), activation='relu'),
                                 keras.layers.AvgPool2D(pool_size = (2,2)),
                                 

                                 keras.layers.Conv2D(128, (3,3), activation='relu'),
                                 keras.layers.AvgPool2D(pool_size = (2,2)),

                                 keras.layers.Conv2D(64, (3,3), activation='relu'),
                                 keras.layers.AvgPool2D(pool_size=(2,2)),

                                 keras.layers.Flatten(),
                                 keras.layers.Dropout(0.5),

                                 keras.layers.Dense(2048, activation='relu'),
                                 keras.layers.Dropout(0.5),
                                 keras.layers.Dense(1024, activation='relu'),
                                 keras.layers.Dropout(0.5),
                                 keras.layers.Dense(2, activation='softmax')

])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5, validation_split=0.2, verbose=1, batch_size=16)

Epoch 1/5
48/48 [==============================] - 10s 209ms/step - loss: 0.2361 - accuracy: 0.9136 - val_loss: 0.2962 - val_accuracy: 0.8691
Epoch 2/5
48/48 [==============================] - 10s 207ms/step - loss: 0.2330 - accuracy: 0.9188 - val_loss: 0.2768 - val_accuracy: 0.9005
Epoch 3/5
48/48 [==============================] - 10s 207ms/step - loss: 0.2459 - accuracy: 0.8966 - val_loss: 0.3587 - val_accuracy: 0.8953
Epoch 4/5
48/48 [==============================] - 10s 207ms/step - loss: 0.2317 - accuracy: 0.9058 - val_loss: 0.2795 - val_accuracy: 0.9215
Epoch 5/5
48/48 [==============================] - 10s 208ms/step - loss: 0.2237 - accuracy: 0.9110 - val_loss: 0.2518 - val_accuracy: 0.9110


In [ ]:
preds = model.predict(x_test)

In [ ]:
predictions = [numpy.argmax(i) for i in preds]

In [ ]:
l = [numpy.argmax(i) for i in y_test]

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(l, predictions)

0.9425465838509317

In [ ]:
final_dictionary = {1:'Mask', 0:'No_Mask'}

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/new mask detection/trained_model.h5')